In [1]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

import nltk
nltk.download('punkt')
import pprint
import re
from lxml import etree
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from gensim.models import Word2Vec
from gensim.models import FastText
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)


auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

id = '1B47OiEiG2Lo1jUY6hy_zMmHBxfKQuJ8-'
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('ted_en-20160408.xml') 

targetXML=open('ted_en-20160408.xml', 'r', encoding='UTF8')

# Getting contents of <content> tag from the xml file
target_text = etree.parse(targetXML)
parse_text = '\n'.join(target_text.xpath('//content/text()'))

# Removing "Sound-effect labels" using regular expression (i.e. (Audio), (Laughter))
content_text = re.sub(r'\([^)]*\)', '', parse_text)

# Tokenising the sentence to process it by using NLTK library
sent_text=sent_tokenize(content_text)

# Removing punctuations and changing all characters to lower case
normalized_text = []
for string in sent_text:
     tokens = re.sub(r"[^a-z0-9]+", " ", string.lower())
     normalized_text.append(tokens)

# Tokenising each sentence to process individual word
sentences=[]
sentences=[word_tokenize(sentence) for sentence in normalized_text]

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [0]:
def get_4_models(sentences, model, architecture, Formula, positiveList, negativeList):

    print('Formula:', Formula)
    print('Model:', model)
    print('Architecture:', architecture)
    print('\nPositive Words:', positiveList)
    print('Negative Words:', negativeList)

    if(model == 'FastText' and architecture == 'Skip Gram'):
      ft_sg_model = FastText(sentences = sentences, size = 100, window = 5, min_count = 5, workers = 4, sg = 1)
      result = ft_sg_model.wv.most_similar(positive = positiveList, negative = negativeList, topn = 1)
    elif(model == 'FastText' and architecture == 'CBOW'):
      ft_cbow_model = FastText(sentences = sentences, size = 100, window = 5, min_count = 5, workers = 4, sg = 0)
      result = ft_cbow_model.wv.most_similar(positive = positiveList, negative = negativeList, topn = 1)
    elif(model == 'Word2Vec' and architecture == 'Skip Gram'):
      wv_sg_model = Word2Vec(sentences = sentences, size = 100, window = 5, min_count = 5, workers = 4, sg = 1)
      result = wv_sg_model.wv.most_similar(positive = positiveList, negative = negativeList, topn = 1)
    elif(model == 'Word2Vec' and architecture == 'CBOW'):
      wv_cbow_model = Word2Vec(sentences = sentences, size = 100, window = 5, min_count = 5, workers = 4, sg = 0)
      result = wv_cbow_model.wv.most_similar(positive = positiveList, negative = negativeList , topn = 1)

    return result


In [0]:
#@title Word Algebra Calculator 

#@markdown Please select the model, architecture and formula to calculate the word algebra

# Get the input
Formula = 'University + Student' #@param {type: "string"}
processed_Formula = Formula.lower()
Model = 'Word2Vec' #@param ['Word2Vec', 'FastText']
Architecture = "Skip Gram" #@param ["CBOW", "Skip Gram"]

# process formula
processed_Formula = processed_Formula.split()

# separate positive and negative words
positiveList = []
negativeList = []

positiveList.insert(0, processed_Formula[0])

for x in range(len(processed_Formula)):
  if(processed_Formula[x] == '+'):
    positiveList.append(processed_Formula[x+1])
  elif(processed_Formula[x] == '-'):
    negativeList.append(processed_Formula[x+1])
    

In [4]:
# Call the function
result = get_4_models(sentences, Model, Architecture, Formula, positiveList, negativeList)

Formula: University + Student
Model: Word2Vec
Architecture: Skip Gram

Positive Words: ['university', 'student']
Negative Words: []


In [5]:
# print the final result
print('Result:', result[0][0])

Result: harvard
